In [1]:
# import spacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.manifold import MDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.feature_selection import chi2
import os, sys
import glob
import gensim
from gensim.corpora import Dictionary
from gensim.similarities import MatrixSimilarity
from gensim.models import ldamodel, doc2vec, LsiModel, LdaModel, CoherenceModel
from gensim.matutils import kullback_leibler, jaccard, hellinger, sparse2full
import nltk
# nltk.download('punkt')
import string
import csv
import math
import statistics
import datetime
from nltk.corpus import stopwords
from nltk.util import ngrams
# nltk.download('stopwords')
from collections import OrderedDict, Counter, namedtuple
import networkx as nx
import matplotlib.pyplot as plt
from networkx.readwrite import json_graph
from bokeh.plotting import figure, show, output_file
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.layouts import row, column
from progress.bar import IncrementalBar
import warnings
warnings.filterwarnings('ignore')
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
# import warnings
# warnings.filterwarnings('ignore')
# import logging
# # logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)
# logger = logging.getLogger()
# logger.setLevel(logging.CRITICAL)

In [2]:
from ast import literal_eval
import itertools


df = pd.read_csv('../scripts/combined_pages_date_binned.csv')
print(len(df))
model_texts = []
def get_texts(rows):
    
    x = literal_eval(rows.values[0])
    t = ' '.join(x)
    model_texts.append(t)
    return t

df_grouped = df.groupby(['date', 'binned'])['token_texts'].apply(get_texts).reset_index()

model_lists = []
def get_lists(rows):

    x = literal_eval(rows.values[0])
#     print(len(x))
    combined = list(itertools.chain.from_iterable(x))

#     print(len(combined))
    model_lists.append(combined)
#     print(len(model_lists))
    return combined


df_grouped1 = df.groupby(['date', 'binned'])['token_lists'].apply(get_lists).reset_index()
print(len(model_texts))


341
341


In [4]:
# test1 = df_grouped1.loc[df_grouped1.year == 1963]
# word_dist = nltk.FreqDist(test1.token_lists.values[0])
# # # word_dist = nltk.FreqDist(frames)

# # # stopwords = nltk.corpus.stopwords.words('english')
# # # words_except_stop_dist = nltk.FreqDist(w for w in words if w not in stopwords) 

# # # print('All frequencies, including STOPWORDS:')
# # # # print('=' * 60)
# rslt = pd.DataFrame(word_dist.most_common(1000),columns=['Word', 'Frequency'])
# # # # plot
# rslt.hist(column='Frequency')
# # # test1.token_lists
# # word_dist.plot()


In [5]:
# tfidf_model = TfidfVectorizer(lowercase=False, max_df=0.30, min_df=0.05)
# tfidf = tfidf_model.fit_transform(model_texts)

# corpus = gensim.matutils.Sparse2Corpus(tfidf, documents_columns=False)
# dictionary = Dictionary.from_corpus(corpus,id2word=dict((id, word) for word, id in tfidf_model.vocabulary_.items()))
# model = LdaModel(corpus=corpus, id2word=dictionary, iterations=50, num_topics=20)

In [8]:
def make_topics_bow(topic):
#     print(topic)
#     # takes the string returned by model.show_topics()
#     # split on strings to get topics and the probabilities
#     topic = topic.split('+')
#     # list to store topic bows
    topic_bow = []
    for word in topic:
#         print(word)
#         # split probability and word
        word, prob = word
#         # get rid of spaces
        word = word.replace(" ","")
#         # convert to word_type
        word = model.id2word.doc2bow([word])[0][0]
        topic_bow.append((word, float(prob)))
    return topic_bow

def make_topics_bow2(topic):
#     print(topic)
#     # takes the string returned by model.show_topics()
#     # split on strings to get topics and the probabilities
#     topic = topic.split('+')
#     # list to store topic bows
    topic_bow = []
    for word in topic:
#         print(word)
#         # split probability and word
        word, prob = word
#         # get rid of spaces
        word = word.replace(" ","")
#         # convert to word_type
        word = model.id2word.doc2bow([word])[0][0]
        topic_bow.append((word, float(prob)))
    return topic_bow
frames = []
num_topics = 20
for t in range(num_topics):
    print(t)
#     if t+1 < num_topics:
#         print(t, t+1)
    test_1 = model.show_topic(t, topn=len(model.id2word))
    finance_distribution = make_topics_bow(test_1)
    for to in range(num_topics):
#             if to+1 < num_topics:
#         print(t, to)
        test_2 = model.show_topic(to, topn=len(model.id2word))


        water_distribution = make_topics_bow(test_2)
        h = hellinger(water_distribution, finance_distribution)
        kl = kullback_leibler(water_distribution, finance_distribution)
        print(str(t), str(to), 'h ', h, kl)
        d = {}
        d['topic_1'] = str(t)
        d['topic_2'] = str(to)
        d['hellinger'] = h
        d['kullback_leibler'] = kl
        dl = pd.DataFrame().append(d, ignore_index=True)
        frames.append(dl)
df_sim = pd.concat(frames)
# df_sim.to_csv('congo_topics_similarities.csv')

0
0 0 h  0.0 0.0
0 1 h  0.40811932681225765 0.72318876
0 2 h  0.41134246854996864 0.73985386
0 3 h  0.3504210059214881 0.54844487
0 4 h  0.4406751887265353 0.83802843
0 5 h  0.43340222116772303 0.7931657
0 6 h  0.41309062906691274 0.7561321
0 7 h  0.3987852321304102 0.688675
0 8 h  0.40788114104628254 0.74604845
0 9 h  0.38512889947826057 0.6727455
0 10 h  0.4073781900852574 0.7237114
0 11 h  0.3715428181737433 0.6230773
0 12 h  0.39968455616414594 0.69823426
0 13 h  0.4344732152729706 0.8284141
0 14 h  0.5102225153760939 1.3500055
0 15 h  0.42636186188534586 0.79720294
0 16 h  0.4319417821134978 0.8122195
0 17 h  0.39971008648187345 0.7055707
0 18 h  0.41888206358903757 0.7733989
0 19 h  0.41824809810592445 0.75981724
1
1 0 h  0.40811932681225765 0.7268147
1 1 h  0.0 0.0
1 2 h  0.4237597738625731 0.79398686
1 3 h  0.36375266111931565 0.5901098
1 4 h  0.4553903604677411 0.9065583
1 5 h  0.4535518945697523 0.88714075
1 6 h  0.4279862625691887 0.81845176
1 7 h  0.41455293645942776 0.7586

11 6 h  0.3754592743757393 0.58632576
11 7 h  0.36916061610657747 0.55463356
11 8 h  0.37453816006204527 0.59407383
11 9 h  0.3483351907804543 0.514406
11 10 h  0.37746126639489996 0.5848525
11 11 h  0.0 0.0
11 12 h  0.3611700289130526 0.541518
11 13 h  0.4005499132992252 0.6624279
11 14 h  0.4775101679778857 1.1189885
11 15 h  0.37775618864431637 0.5857715
11 16 h  0.3928364146414705 0.630483
11 17 h  0.3656392866178963 0.5520258
11 18 h  0.40176200307410453 0.66719985
11 19 h  0.40571640065266246 0.6781042
12
12 0 h  0.39968455616414594 0.6851881
12 1 h  0.4058543615306295 0.7033448
12 2 h  0.410982177281927 0.72503847
12 3 h  0.34926140575032705 0.5386605
12 4 h  0.432673774653115 0.801799
12 5 h  0.4261819824361639 0.7616417
12 6 h  0.40841139168175294 0.7225641
12 7 h  0.39529177761606776 0.66551614
12 8 h  0.4005732128714751 0.7125094
12 9 h  0.36991011678217994 0.610464
12 10 h  0.4034506414194234 0.70163345
12 11 h  0.3611700289130526 0.5768739
12 12 h  0.0 0.0
12 13 h  0.43454

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from itertools import combinations 
sim = df_sim.loc[(df_sim.topic_1 == df_sim.topic_2) == False]
sim.sort_values(by=['topic_1'])

# kl = sim.drop(['hellinger'], axis=1)
# sim.kullback_leibler = sim.kullback_leibler.round(2)
# sim.hellinger = sim.hellinger.round(2)
sim['avg'] = sim[['hellinger', 'kullback_leibler']].mean(axis=1)
sim.avg = sim.avg.round(2)
sim['top_1'] = 'top' + sim.topic_1.astype(str)
# # sum_combos.year = sum_combos.year.astype(int)
pi = pd.pivot_table(sim, values='avg', index=['topic_1'], columns=['topic_2'], fill_value=0, aggfunc=np.sum).reset_index()
# pi.sortlevel(level=0, ascending=True, inplace=True)
pi.sort_values(by=['topic_1'], inplace=True)
pi.sort_index(axis=1, inplace=True)
pi.reset_index()
pi.drop(['topic_1'], axis=1, inplace=True)
# # # # plt.pcolor(pi)
# # # # plt.yticks(np.arange(0.5, len(pi.index), 1), pi.index)
# # # # plt.xticks(np.arange(0.5, len(pi.columns), 1), pi.columns)
plt.figure(figsize=(20, 26))
ax = plt.axes()
g = sns.heatmap(pi, ax=ax, cmap="YlGnBu", linewidths=.5)
ax.set_title('Topic similarities for Congo keywords', fontsize=16)
# ax.xaxis.tick_top()
# # # # # # axes = g.axes.flatten()
# # # # # # axes[0].set_title('Correlation between Congo Crisis Keywords and Corpus Distributions over Time')
ax.set_ylabel('')
ax.set_xlabel('')
# # ax.tick_params(labelsize=14)

plt.savefig('congo_topic_similarity.png')
plt.show()


In [213]:
model.show_topics(num_topics=20,formatted=False,num_words=10)

[(0,
  [('thant', 0.0014006003),
   ('art', 0.0012467573),
   ('fishing', 0.001171364),
   ('wall', 0.0010610113),
   ('station', 0.0010127807),
   ('kr', 0.0009318233),
   ('ruanda', 0.0009299084),
   ('bureau', 0.0009242717),
   ('urundi', 0.00087537157),
   ('gizenga', 0.00085291365)]),
 (1,
  [('ecm', 0.0015870228),
   ('buganda', 0.0011017353),
   ('roberto', 0.0009833534),
   ('dollars', 0.0009697605),
   ('lake', 0.0008970762),
   ('kingdom', 0.00087667664),
   ('embassy', 0.0008726141),
   ('worth', 0.00082591426),
   ('heikal', 0.0008094485),
   ('tne', 0.0007790772)]),
 (2,
  [('team', 0.0013620945),
   ('disarmament', 0.0013445306),
   ('mobutu', 0.0012904585),
   ('johnson', 0.001233153),
   ('burundi', 0.0012056812),
   ('gurion', 0.0011448266),
   ('jews', 0.0011323974),
   ('atomic', 0.0011229843),
   ('nehru', 0.0011203319),
   ('belgrade', 0.0010890865)]),
 (3,
  [('bank', 0.0014364455),
   ('england', 0.0013508752),
   ('bella', 0.0013322211),
   ('health', 0.00129160

In [214]:
cm_mass = CoherenceModel(model=model, corpus=corpus, dictionary=dictionary, coherence='u_mass')

In [215]:
print('mass', cm_mass.get_coherence())

mass -2.6617459745328524


In [39]:
cm_cv = CoherenceModel(model=model, corpus=corpus, dictionary=dictionary, coherence='c_v')
print('cv', cm_cv.get_coherence())

ValueError: ("'texts' should be provided for %s coherence.", 'c_v')

In [42]:
len(model_lists)

7

In [3]:
dictionary_zero = Dictionary(model_lists)
dictionary_zero.filter_extremes(no_above=0.2)
corpus_zero = [dictionary_zero.doc2bow(text) for text in model_lists]

In [4]:
model = LdaModel(corpus=corpus_zero, id2word=dictionary_zero, iterations=400, num_topics=20, alpha='auto', eta='auto', passes=20)

/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fro

/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fro

/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fro

/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fro

/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fro

...

/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fro

/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fro

/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fro

/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fro

/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/zgl3n/.local/share/virtualenvs/CRDH-R-3_7VyR/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fro

In [375]:
cm_cv_zero = CoherenceModel(model=model, texts=model_lists, dictionary=dictionary_zero, coherence='c_v')

In [376]:
cm_cv_zero.get_coherence()

0.3309543063961652

In [225]:
model_zero.show_topics(num_topics=20,formatted=False,num_words=10)

[(0,
  [('lumumba', 0.004391382),
   ('—', 0.0043870206),
   ('paper', 0.0028354647),
   ('angola', 0.0027278136),
   ('israeli', 0.0025525333),
   ('sudan', 0.0024243833),
   ('leopoldville', 0.0023547653),
   ('gaulle', 0.0023446362),
   ('hammarskjoeld', 0.0022899904),
   ('radio', 0.0022753943)]),
 (1,
  [('—', 0.0049109366),
   ('lumumba', 0.0041348897),
   ('sudan', 0.004004139),
   ('..', 0.0038946113),
   ('israeli', 0.0038515471),
   ('trade', 0.003108858),
   ('portuguese', 0.0028066952),
   ('soviet', 0.0025711623),
   ('radio', 0.0024636008),
   ('federation', 0.0023885022)]),
 (2,
  [('—', 0.0055074636),
   ('lumumba', 0.005110132),
   ('..', 0.004569962),
   ('angola', 0.0025557748),
   ('trade', 0.0025096084),
   ('``', 0.0024507209),
   ('paper', 0.002338649),
   ('children', 0.002234144),
   ('capital', 0.0022339164),
   ('cyprus', 0.0021354337)]),
 (3,
  [('—', 0.0061127613),
   ('lumumba', 0.0029818525),
   ('egypt', 0.002577531),
   ('king', 0.002551733),
   ('resol

In [8]:
import pyLDAvis
# import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings('ignore')
import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
def create_models(texts):
#     tfidf_model = TfidfVectorizer(lowercase=False, max_df=0.20)
#     tfidf = tfidf_model.fit_transform(texts)

#     corpus = gensim.matutils.Sparse2Corpus(tfidf, documents_columns=False)
#     dictionary = Dictionary.from_corpus(corpus,id2word=dict((id, word) for word, id in tfidf_model.vocabulary_.items()))
#     lda = LdaModel(corpus=corpus, id2word=dictionary, iterations=400, num_topics=20, alpha='auto', eta='auto', eval_every=1, passes=20)
    dictionary = Dictionary(texts)
    dictionary.filter_extremes(no_above=0.2)
    corpus = [dictionary.doc2bow(text) for text in texts]
    lda = LdaModel(corpus=corpus, id2word=dictionary, iterations=400, num_topics=20, alpha='auto', eta='auto', passes=20)
#     print('Number of unique tokens: %d' % len(dictionary))
#     print('Number of documents: %d' % len(corpus))
    vis_data = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
    return vis_data

# docs = []
# for doc in documents:
#     d = doc.split(' ')
#     docs.append(d)
vis_data = create_models(model_lists)
# create_models(model_lists)


pyLDAvis.display(vis_data)

In [252]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model=LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, iterations=400, alpha='auto', eta='auto', passes=20)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values
limit=150; start=2; step=10;
x = range(start, limit, step)
model_list, coherence_values = compute_coherence_values(dictionary=dictionary_zero, corpus=corpus_zero, texts=model_lists, start=start, limit=limit, step=step)
# # # Show graph

plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

KeyboardInterrupt: 

In [138]:
df = pd.read_csv('../scripts/combined_pages_date_congo_text_ner.csv')
len(df)

110

In [345]:
df = pd.read_csv('../data/combined_all_data_ner_congo.csv')
labels = [0,1,2,3,4]
df_1963 = df.loc[df.year == 1963]

df_1963['binned'] = pd.qcut(df_1963.page_number, 5, labels=labels)
len(df_1963)

294

In [346]:
labels = [0,1,2,3,4]
df_1964 = df.loc[df.year == 1964]

# len(df_1964.page_number.unique())
df_1964['binned'] = pd.qcut(df_1964.page_number, 5, labels=labels)

In [347]:
labels = [0,1,2,3,4]
df_1965 = df.loc[df.year == 1965]

# len(df_1965.page_number.unique())
df_1965['binned'] = pd.qcut(df_1965.page_number, 5, labels=labels)

In [348]:
labels = [0,1]
df_1966 = df.loc[df.year == 1966]

# len(df_1966.page_number.unique())
df_1966['binned'] = pd.qcut(df_1966.page_number, 2, labels=labels)

In [349]:
df_1960 = df.loc[df.year == 1960]
df_1960['binned'] = 0
df_1961 = df.loc[df.year == 1961]
df_1961['binned'] = 0
df_1962 = df.loc[df.year == 1962]
df_1962['binned'] = 0

In [350]:
frames = [df_1960, df_1961, df_1962, df_1963, df_1964, df_1965, df_1966]

In [352]:
df = pd.concat(frames)
df.date.unique()

array(['1960-August-14', '1960-August-7', '1960-December-25',
       '1960-December-4', '1960-July-10', '1960-July-17', '1960-July-3',
       '1960-June-26', '1960-November-13', '1960-November-6',
       '1960-October-16', '1960-October-30', '1960-September-11',
       '1961-April-16', '1961-April-23', '1961-April-2', '1961-April-30',
       '1961-April-9', '1961-August-13', '1961-August-20',
       '1961-August-27', '1961-December-11', '1961-December-18',
       '1961-December-25', '1961-February-26', '1961-January-15',
       '1961-January-22', '1961-July-16', '1961-July-23', '1961-July-30',
       '1961-June-11', '1961-June-18', '1961-June-25', '1961-June-4',
       '1961-March-12', '1961-March-19', '1961-March-5', '1961-May-14',
       '1961-May-21', '1961-May-7', '1961-November-13',
       '1961-November-20', '1961-November-27', '1961-October-15',
       '1961-October-1', '1961-October-29', '1961-October-8',
       '1961-September-10', '1961-September-24', '1961-September-3',
    

In [353]:
df.to_csv('combined_all_ner_congo_data_binned.csv')

In [256]:
len(df)

128

In [200]:
def join_token_lists(rows):
    texts = rows
    final_doc = []
#     sentences = []
#     for line in texts:
#         sentences.extend(nltk.sent_tokenize(line))
#     for sentence in sentences:
#         sent_tokens = [token.lower() for token in nltk.word_tokenize(sentence) if token not in string.punctuation and token not in stopwords.words('english') and token.isdigit() == False]


#         final_doc.append(sent_tokens)
    return final_doc

def get_texts(rows):

    texts = rows.tolist()
    return texts[0]

df_tokenized = d.groupby(['date', 'binned', 'page_number'])['tokenized_text'].apply(get_texts).reset_index()
df_lists = df_tokenized.groupby(['date', 'binned'])['tokenized_text'].apply(join_token_lists).reset_index()
print(df_lists)

                date  binned tokenized_text
0        1966-May-01       1             []
1  1966-September-01       0             []
2  1966-September-01       1             []


In [356]:
d = pd.read_csv('../scripts/combined_pages_date_congo_text_ner_binned.csv')
d.date.unique()

array(['1960-August-14', '1960-August-7', '1960-December-25',
       '1960-December-4', '1960-July-10', '1960-July-17', '1960-July-3',
       '1960-June-26', '1960-November-13', '1960-November-6',
       '1960-October-16', '1960-October-30', '1960-September-11',
       '1961-April-16', '1961-April-2', '1961-April-23', '1961-April-30',
       '1961-April-9', '1961-August-13', '1961-August-20',
       '1961-August-27', '1961-December-11', '1961-December-18',
       '1961-December-25', '1961-February-26', '1961-January-15',
       '1961-January-22', '1961-July-16', '1961-July-23', '1961-July-30',
       '1961-June-11', '1961-June-18', '1961-June-25', '1961-June-4',
       '1961-March-12', '1961-March-19', '1961-March-5', '1961-May-14',
       '1961-May-21', '1961-May-7', '1961-November-13',
       '1961-November-20', '1961-November-27', '1961-October-1',
       '1961-October-15', '1961-October-29', '1961-October-8',
       '1961-September-10', '1961-September-24', '1961-September-3',
    

In [ ]:
dictionary = Dictionary(texts)
dictionary.filter_extremes(no_above=0.2)
corpus = [dictionary.doc2bow(text) for text in texts]
lda = LdaModel(corpus=corpus, id2word=dictionary, iterations=400, num_topics=50, alpha='auto', eta='auto', passes=20)